In [4]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

##  Работа с текстовыми данными 
Будем работать с данными с семинара, но использовать все категории текстов:

In [5]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True,random_state=7)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True,random_state=7)#

In [6]:
X_train=twenty_train.data
X_test=twenty_test.data
y_train=twenty_train.target
y_test=twenty_test.target

### 1. Мешок слов (1 балл)
Воспользуйтесь мешком слов  для признакового описания документа. Постройте логистическую регрессию с параметрами по умолчанию.  Какова доля верных предсказаний на тестовой выборке?

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [8]:
count_vect = CountVectorizer() 

X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_counts, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
y_pred_counts = lr.predict(X_test_counts)
print ('Accuracy =', metrics.accuracy_score(y_test, y_pred_counts))

Accuracy = 0.8069569835369091


### 2. TF-IDF (1 балл)
Теперь в качестве признаков возьмите величины tf-idf. Как поменялось качество классификатора?

In [10]:
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

lr.fit(X_train_tfidf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
y_pred_tfidf = lr.predict(X_test_tfidf)
print ('Accuracy =', metrics.accuracy_score(y_test, y_pred_tfidf))

Accuracy = 0.8279341476367499


### 3.  Матрица ошибок (1 балл)
Выведите матрицу ошибок. Определите три пары категорий, которые модель путает чаще всего.  

In [12]:
cm = metrics.confusion_matrix(y_test,y_pred_tfidf)
print(cm)

[[236   2   0   0   1   1   3   0   3   3   1   1   2   9   2  35   3   4
    1  12]
 [  2 304  13   8   8  19   7   1   0   1   0   4  10   1   5   2   1   1
    1   1]
 [  1  21 294  31  12  10   2   2   1   5   0   1   1   2   4   2   0   0
    4   1]
 [  0  12  24 283  22   2  13   3   1   1   1   1  24   0   4   0   0   0
    0   1]
 [  0   5   6  21 319   1  11   1   1   5   1   0  10   0   1   0   0   0
    3   0]
 [  0  41  38   5   4 292   3   0   1   1   0   1   3   2   4   0   0   0
    0   0]
 [  0   3   2  12   7   0 351   3   2   1   1   0   7   1   0   0   0   0
    0   0]
 [  0   1   1   4   0   2  12 352   4   2   0   0  13   1   1   0   1   0
    2   0]
 [  0   0   0   0   0   0   6  10 379   2   0   0   1   0   0   0   0   0
    0   0]
 [  1   0   0   0   2   0   6   1   0 368  15   0   2   0   0   0   1   0
    0   1]
 [  0   0   0   1   3   1   2   0   0   9 382   0   0   0   0   0   0   0
    1   0]
 [  1   7   3   0   3   4   5   2   1   5   0 352   5   0   1   0

In [13]:
maximum1 = 0
max1_i=0
max1_j=0
for i in range(cm.shape[0]):
    for j in range (cm.shape[1]):
        if (i != j):
            if (cm[i][j] > maximum1):
                maximum1= cm[i][j]
                max1_i=i
                max1_j=j

print(maximum1, max1_i, max1_j)
        
        

97 18 16


In [14]:
maximum2 = 0
max2_i=0
max2_j=0
for i in range(cm.shape[0]):
    for j in range (cm.shape[1]):
        if (i != j ):
            if (cm[i][j] > maximum2 and cm[i][j] != maximum1 ):
                maximum2= cm[i][j]
                max2_i=i
                max2_j=j

print(maximum2, max2_i, max2_j)

48 19 15


In [15]:
maximum3 = 0
max3_i=0
max3_j=0
for i in range(cm.shape[0]):
    for j in range (cm.shape[1]):
        if (i != j ):
            if (cm[i][j] > maximum3 and cm[i][j] != maximum1 and cm[i][j] != maximum2  ):
                maximum3= cm[i][j]
                max3_i=i
                max3_j=j

print(maximum3, max3_i, max3_j)

41 5 1


##### Чаще всего алгоритм путает следующие три пары категорий: 18-16, 19-15, 5-1  
(Наибольше недиагональные элементы матрицы ошибок)

### 4.Определение наиболее важных признаков (слов) (2 балла)
Выберите категорию. Для гиперплоскости, отделяющей класс этой категории от остальных, определите 5 признаков, которые имеют наибольшие коэффициенты в линейной модели и 5 признаков, которые имеют наименьшие коэффициенты:
* определите слова, котором соответствуют наибольшие и наименьшие коэффициенты в линейной модели;
* напишите  функцию поиска ключа в словаре по заданному листу значений. 

In [16]:
w = lr.coef_[0] #Выбрали нулевую категорию
w = pd.DataFrame({'Feature name': list(count_vect.get_feature_names()),'Feature weight':w},columns = ['Feature name','Feature weight'])

w_max = w.sort_values(by='Feature weight',ascending=False).head(5) # 5 слов с наибольшими коэффициентами 
w_max

,Feature name,Feature weight
72154,keith,4.832775
30101,atheism,4.167555
30105,atheists,3.808081
37469,caltech,3.024540
68620,islamic,2.973565


In [17]:
w_min= w.sort_values(by='Feature weight',ascending=True).head(5) # 5 слов с наименьшими коэффициентами 
w_min

,Feature name,Feature weight
37219,ca,-1.306474
124616,with,-0.960020
124434,windows,-0.915352
119737,use,-0.900256
119714,usa,-0.855127
